[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_22/uebung_neuronale_netze_digits.ipynb)

# Übung: Neuronale Netze mit Digits Dataset
## MLP vs. CNN Vergleich

In dieser Übung schauen wir uns zwei Arten von neuronalen Netzen an:
- **Multi-Layer Perceptron (MLP)**: Klassisches neuronales Netz mit vollverbundenen Schichten
- **Convolutional Neural Network (CNN)**: Speziell für Bilddaten entwickelt

Wir verwenden den **Digits Dataset**: 8×8 Pixel Bilder handgeschriebener Ziffern (0-9).

## 1. Import Required Libraries

In [ ]:
# Standard Libraries
import numpy as np
import matplotlib.pyplot as plt
import random

# Scikit-learn für Digits Dataset und Metriken
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# TensorFlow/Keras für beide Modelle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print("Bibliotheken geladen!")
print(f"TensorFlow Version: {tf.__version__}")

# Reproducibility - alle Seeds setzen für vollständige Reproduzierbarkeit
SEED = 42
random.seed(SEED)           # Python built-in random
np.random.seed(SEED)        # NumPy random
tf.random.set_seed(SEED)    # TensorFlow random

# Zusätzliche TensorFlow Determinismus-Einstellungen für GPU
import os
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

print(f"Alle Random Seeds auf {SEED} gesetzt für Reproduzierbarkeit!")

## 2. Load and Explore the Digits Dataset

In [ ]:
# Digits Dataset laden
digits = load_digits()
X, y = digits.data, digits.target

print("Dataset Informationen:")
print(f"Anzahl Bilder: {X.shape[0]}")
print(f"Features pro Bild: {X.shape[1]} (8×8 = 64 Pixel)")
print(f"Bildgröße: 8×8 Pixel")
print(f"Anzahl Klassen: {len(np.unique(y))}")
print(f"Klassen: {np.unique(y)}")
print(f"Pixel-Werte: {X.min()} bis {X.max()}")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape[0]} Bilder")
print(f"Test set: {X_test.shape[0]} Bilder")

# Einige Beispielbilder anzeigen
fig, axes = plt.subplots(2, 5, figsize=(12, 6))
for i, ax in enumerate(axes.flat):
    # Reshape zu 8x8 für Visualisierung
    image = X_train[i].reshape(8, 8)
    ax.imshow(image, cmap='gray')
    ax.set_title(f'Label: {y_train[i]}')
    ax.axis('off')
plt.suptitle('Beispielbilder aus dem Digits Dataset (8×8 Pixel)')
plt.tight_layout()
plt.show()

## 3. Data Preprocessing

In [ ]:
# Daten normalisieren (bereits 0-16 Bereich, normalisieren zu 0-1)
X_train = X_train.astype('float32') / 16.0
X_test = X_test.astype('float32') / 16.0

print(f"Training set: {X_train.shape[0]} Bilder")
print(f"Test set: {X_test.shape[0]} Bilder")
print(f"Normalisierte Pixel-Werte: {X_train.min():.2f} bis {X_train.max():.2f}")

# Daten für MLP: bereits flach (64 Features)
X_train_mlp = X_train.copy()
X_test_mlp = X_test.copy()

# Daten für CNN: Reshape zu 8x8 + Channel-Dimension
X_train_cnn = X_train.reshape(-1, 8, 8, 1)
X_test_cnn = X_test.reshape(-1, 8, 8, 1)

print(f"\nMLP Input Shape: {X_train_mlp.shape}")
print(f"CNN Input Shape: {X_train_cnn.shape}")

# Visualisierung des Unterschieds
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# MLP Sicht: Flacher Vektor
axes[0].bar(range(64), X_train_mlp[0])
axes[0].set_title('MLP Sicht: 64-dimensionaler Vektor')
axes[0].set_xlabel('Pixel Index')
axes[0].set_ylabel('Pixel Wert')

# CNN Sicht: 8x8 Bild
axes[1].imshow(X_train_cnn[0].reshape(8, 8), cmap='gray')
axes[1].set_title('CNN Sicht: 8×8 Bild')
axes[1].axis('off')

plt.suptitle(f'Gleiche Ziffer ({y_train[0]}), verschiedene Darstellungen')
plt.tight_layout()
plt.show()

## 4. Multi-Layer Perceptron (MLP) Implementation

**Wichtiger Hinweis:** Für einen fairen Vergleich zwischen MLP und CNN verwenden wir Modelle mit ähnlicher Parameter-Anzahl. Das verhindert, dass ein Modell nur wegen mehr Parametern besser abschneidet.

In [ ]:
# MLP mit Keras
print("Aufbau MLP Modell...")

mlp_model = keras.Sequential([
    keras.Input(shape=(64,)),
    layers.Dense(20, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(10, activation='softmax')
])

mlp_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),  # Gleiche LR wie CNN
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("MLP Architektur:")
mlp_model.summary()
print(f"MLP Parameter: {mlp_model.count_params():,}")

In [ ]:
# MLP trainieren
print("Training MLP...")
mlp_history = mlp_model.fit(
    X_train_mlp, y_train,
    epochs=100,  # Erhöht auf 100 Epochen
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# MLP evaluieren
mlp_loss, mlp_accuracy = mlp_model.evaluate(X_test_mlp, y_test, verbose=0)
print(f"\nMLP Test Genauigkeit: {mlp_accuracy:.4f}")

# Vorhersagen für späteren Vergleich
y_pred_mlp = np.argmax(mlp_model.predict(X_test_mlp, verbose=0), axis=1)

## 5. Convolutional Neural Network (CNN) Implementation

**Architektur-Überlegungen:** Unser CNN ist jetzt so gestaltet, dass es **weniger Parameter** als das MLP hat. Damit können wir prüfen, ob CNNs auch mit weniger Parametern durch ihre spezialisierte Architektur einen Vorteil bei Bilddaten haben.

In [ ]:
# CNN mit Keras - verbesserte Architektur für bessere Performance
print("Aufbau CNN Modell...")

cnn_model = keras.Sequential([
    # Expliziter Input Layer (Keras 3.x Style)
    keras.Input(shape=(8, 8, 1)),
    # Convolutional Block
    layers.Conv2D(20, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(5, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.1),
    layers.Dense(10, activation='softmax')
])

# Optimizer mit reduzierter Learning Rate
cnn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("CNN Architektur:")
cnn_model.summary()
print(f"CNN Parameter: {cnn_model.count_params():,}")

# Parameter-Vergleich
mlp_params = mlp_model.count_params()
cnn_params = cnn_model.count_params()
print(f"\n=== PARAMETER-VERGLEICH ===")
print(f"MLP Parameter: {mlp_params:,}")
print(f"CNN Parameter: {cnn_params:,}")
print(f"Verhältnis CNN/MLP: {cnn_params/mlp_params:.2f}×")
print(f"Unterschied: {abs(cnn_params - mlp_params):,} Parameter")
if cnn_params < mlp_params:
    print(f"✅ CNN hat {mlp_params - cnn_params:,} Parameter weniger als MLP ({100*(mlp_params-cnn_params)/mlp_params:.1f}% weniger)")
else:
    print(f"📈 CNN hat {cnn_params - mlp_params:,} Parameter mehr als MLP ({100*(cnn_params-mlp_params)/mlp_params:.1f}% mehr)")


In [ ]:
# CNN trainieren
print("Training CNN...")
cnn_history = cnn_model.fit(
    X_train_cnn, y_train,
    epochs=100,  # Erhöht auf 100 Epochen
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# CNN evaluieren
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test_cnn, y_test, verbose=0)
print(f"\nCNN Test Genauigkeit: {cnn_accuracy:.4f}")

# Vorhersagen für Vergleich
y_pred_cnn = np.argmax(cnn_model.predict(X_test_cnn, verbose=0), axis=1)

## 6. Model Comparison and Evaluation

In [ ]:
# Vergleich der Modelle
print("=== MODELL VERGLEICH ===")
print(f"MLP Genauigkeit:  {mlp_accuracy:.4f}")
print(f"CNN Genauigkeit:  {cnn_accuracy:.4f}")
print(f"Unterschied CNN-MLP:     {cnn_accuracy - mlp_accuracy:.4f}")

# Parameter-Anzahl vergleichen
mlp_params = mlp_model.count_params()
cnn_params = cnn_model.count_params()
print(f"\nMLP Parameter:    {mlp_params:,}")
print(f"CNN Parameter:    {cnn_params:,}")
print(f"Verhältnis CNN/MLP: {cnn_params/mlp_params:.1f}×")

# Training History Visualisierung
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(mlp_history.history['loss'], label='MLP Training', alpha=0.7)
plt.plot(mlp_history.history['val_loss'], label='MLP Validation', alpha=0.7)
plt.plot(cnn_history.history['loss'], label='CNN Training', alpha=0.7)
plt.plot(cnn_history.history['val_loss'], label='CNN Validation', alpha=0.7)
plt.title('Training Loss Vergleich')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 2)
plt.plot(mlp_history.history['accuracy'], label='MLP Training', alpha=0.7)
plt.plot(mlp_history.history['val_accuracy'], label='MLP Validation', alpha=0.7)
plt.plot(cnn_history.history['accuracy'], label='CNN Training', alpha=0.7)
plt.plot(cnn_history.history['val_accuracy'], label='CNN Validation', alpha=0.7)
plt.title('Training Accuracy Vergleich')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 3, 3)
models = ['MLP', 'CNN']
accuracies = [mlp_accuracy, cnn_accuracy]
colors = ['skyblue', 'lightcoral']
bars = plt.bar(models, accuracies, color=colors)
plt.title('Test Accuracy Vergleich')
plt.ylabel('Accuracy')
plt.ylim(0.8, 1.0)
for i, (acc, bar) in enumerate(zip(accuracies, bars)):
    plt.text(bar.get_x() + bar.get_width()/2., acc + 0.005, 
             f'{acc:.3f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Konfusionsmatrizen vergleichen
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# MLP Konfusionsmatrix
cm_mlp = confusion_matrix(y_test, y_pred_mlp)
im1 = axes[0].imshow(cm_mlp, interpolation='nearest', cmap='Blues')
axes[0].set_title(f'MLP Confusion Matrix\nAccuracy: {mlp_accuracy:.3f}')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')
axes[0].set_xticks(range(10))
axes[0].set_yticks(range(10))

# CNN Konfusionsmatrix
cm_cnn = confusion_matrix(y_test, y_pred_cnn)
im2 = axes[1].imshow(cm_cnn, interpolation='nearest', cmap='Blues')
axes[1].set_title(f'CNN Confusion Matrix\nAccuracy: {cnn_accuracy:.3f}')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')
axes[1].set_xticks(range(10))
axes[1].set_yticks(range(10))

# Zahlen in die Matrizen schreiben
for i in range(10):
    for j in range(10):
        if cm_mlp[i, j] > 0:
            axes[0].text(j, i, str(cm_mlp[i, j]), ha='center', va='center', 
                        color='white' if cm_mlp[i, j] > cm_mlp.max()/2 else 'black', fontsize=10)
        if cm_cnn[i, j] > 0:
            axes[1].text(j, i, str(cm_cnn[i, j]), ha='center', va='center',
                        color='white' if cm_cnn[i, j] > cm_cnn.max()/2 else 'black', fontsize=10)

plt.colorbar(im1, ax=axes[0])
plt.colorbar(im2, ax=axes[1])
plt.tight_layout()
plt.show()

# Detaillierte Klassifikationsberichte
print("\n=== MLP Classification Report ===")
print(classification_report(y_test, y_pred_mlp))

print("\n=== CNN Classification Report ===")
print(classification_report(y_test, y_pred_cnn))

In [ ]:
# Beispiel-Vorhersagen visualisieren - gezielt Fehler zeigen
fig, axes = plt.subplots(3, 8, figsize=(16, 8))

# Fehleranalyse für gezielte Auswahl
mlp_errors = (y_pred_mlp != y_test)
cnn_errors = (y_pred_cnn != y_test)
both_correct = (~mlp_errors) & (~cnn_errors)  # Beide richtig
mlp_only_wrong = mlp_errors & (~cnn_errors)   # Nur MLP falsch
cnn_only_wrong = (~mlp_errors) & cnn_errors   # Nur CNN falsch
both_wrong = mlp_errors & cnn_errors          # Beide falsch

print("=== FEHLERVERTEILUNG ===")
print(f"Beide richtig: {both_correct.sum()}")
print(f"Nur MLP falsch: {mlp_only_wrong.sum()}")
print(f"Nur CNN falsch: {cnn_only_wrong.sum()}")
print(f"Beide falsch: {both_wrong.sum()}")

# Gezielt interessante Beispiele auswählen
selected_indices = []

# 2 Beispiele wo beide richtig sind
if both_correct.sum() >= 2:
    both_correct_indices = np.where(both_correct)[0]
    selected_indices.extend(both_correct_indices[:2])

# 2 Beispiele wo nur MLP falsch ist (CNN besser)
if mlp_only_wrong.sum() >= 2:
    mlp_only_wrong_indices = np.where(mlp_only_wrong)[0]
    selected_indices.extend(mlp_only_wrong_indices[:2])

# 2 Beispiele wo nur CNN falsch ist (MLP besser)  
if cnn_only_wrong.sum() >= 2:
    cnn_only_wrong_indices = np.where(cnn_only_wrong)[0]
    selected_indices.extend(cnn_only_wrong_indices[:2])

# 2 Beispiele wo beide falsch sind
if both_wrong.sum() >= 2:
    both_wrong_indices = np.where(both_wrong)[0]
    selected_indices.extend(both_wrong_indices[:2])

# Falls nicht genug Beispiele, fülle mit zufälligen auf
while len(selected_indices) < 8:
    remaining_indices = np.setdiff1d(np.arange(len(y_test)), selected_indices)
    if len(remaining_indices) > 0:
        selected_indices.append(np.random.choice(remaining_indices))
    else:
        break

# Auf 8 Beispiele beschränken
selected_indices = selected_indices[:8]

for i, idx in enumerate(selected_indices):
    # Originalbild
    axes[0, i].imshow(X_test_cnn[idx].reshape(8, 8), cmap='gray')
    axes[0, i].set_title(f'True: {y_test[idx]}', fontsize=12, fontweight='bold')
    axes[0, i].axis('off')
    
    # MLP Vorhersage
    mlp_pred = y_pred_mlp[idx]
    mlp_correct = (mlp_pred == y_test[idx])
    mlp_color = 'green' if mlp_correct else 'red'
    axes[1, i].text(0.5, 0.5, f'MLP\n{mlp_pred}', transform=axes[1, i].transAxes,
                   ha='center', va='center', fontsize=16, color=mlp_color, weight='bold')
    axes[1, i].axis('off')
    
    # CNN Vorhersage
    cnn_pred = y_pred_cnn[idx]
    cnn_correct = (cnn_pred == y_test[idx])
    cnn_color = 'green' if cnn_correct else 'red'
    axes[2, i].text(0.5, 0.5, f'CNN\n{cnn_pred}', transform=axes[2, i].transAxes,
                   ha='center', va='center', fontsize=16, color=cnn_color, weight='bold')
    axes[2, i].axis('off')
    
    # Kategorisierung als Subtitle
    if mlp_correct and cnn_correct:
        category = "Beide ✓"
        category_color = 'green'
    elif not mlp_correct and cnn_correct:
        category = "CNN besser"
        category_color = 'blue'
    elif mlp_correct and not cnn_correct:
        category = "MLP besser"
        category_color = 'orange'
    else:
        category = "Beide ✗"
        category_color = 'red'
    
    axes[0, i].text(0.5, -0.1, category, transform=axes[0, i].transAxes,
                   ha='center', va='top', fontsize=10, color=category_color, weight='bold')

axes[0, 0].set_ylabel('Original\n(8×8)', fontsize=14)
axes[1, 0].set_ylabel('MLP\nVorhersage', fontsize=14)
axes[2, 0].set_ylabel('CNN\nVorhersage', fontsize=14)
plt.suptitle('Gezielte Beispiele: Grün = Richtig, Rot = Falsch\nKategorien zeigen welches Modell besser abschneidet', fontsize=16)
plt.tight_layout()
plt.show()

# Fehleranalyse: Wo macht das CNN weniger Fehler als das MLP?
cnn_better = mlp_errors & ~cnn_errors  # MLP falsch, CNN richtig
mlp_better = cnn_errors & ~mlp_errors  # CNN falsch, MLP richtig

print(f"\n=== DETAILLIERTE FEHLERANALYSE ===")
print(f"MLP Fehler: {mlp_errors.sum()}/{len(y_test)} ({100*mlp_errors.mean():.1f}%)")
print(f"CNN Fehler: {cnn_errors.sum()}/{len(y_test)} ({100*cnn_errors.mean():.1f}%)")
print(f"CNN besser als MLP: {cnn_better.sum()} Fälle")
print(f"MLP besser als CNN: {mlp_better.sum()} Fälle")
print(f"Beide richtig: {both_correct.sum()} Fälle")
print(f"Beide falsch: {both_wrong.sum()} Fälle")

# Zeige konkrete Beispiele wo CNN besser ist
if cnn_better.sum() > 0:
    print(f"\n📊 Beispiele wo CNN richtig liegt, MLP aber falsch:")
    cnn_better_indices = np.where(cnn_better)[0][:5]  # Erste 5 Beispiele
    for idx in cnn_better_indices:
        print(f"  Index {idx}: True={y_test[idx]}, MLP={y_pred_mlp[idx]} ❌, CNN={y_pred_cnn[idx]} ✅")

# Zeige konkrete Beispiele wo MLP besser ist
if mlp_better.sum() > 0:
    print(f"\n📊 Beispiele wo MLP richtig liegt, CNN aber falsch:")
    mlp_better_indices = np.where(mlp_better)[0][:5]  # Erste 5 Beispiele
    for idx in mlp_better_indices:
        print(f"  Index {idx}: True={y_test[idx]}, MLP={y_pred_mlp[idx]} ✅, CNN={y_pred_cnn[idx]} ❌")

## 🎯 Fazit und Erkenntnisse

**Wichtige Beobachtungen bei 8×8 Bildern und vergleichbaren Parameter-Anzahlen:**

1. **Fairer Vergleich mit ähnlichen Parameter-Zahlen**: CNN erreicht sogar mit ca. 20% weniger Parametern vergleichbare Ergebnisse zum MLP.

2. **Überraschung bei kleinen Bildern**: Bei 8×8 Bildern ist der Unterschied zwischen MLP und CNN oft minimal - manchmal ist das MLP sogar gleichwertig oder besser!

3. **Warum ist das so?**
   - Bei nur 64 Pixeln ist die "räumliche Struktur" weniger komplex
   - Ein kleines MLP kann die wenigen relevanten Pixel-Kombinationen direkt lernen
   - Die Translation-Invarianz von CNNs ist bei zentrierten 8×8 Ziffern weniger wichtig
   - Faltung bringt weniger Vorteil bei so kleinen rezeptiven Feldern

4. **CNN-Vorteile trotzdem sichtbar:**
   - Leicht bessere Robustheit gegen Verschiebungen
   - Parameter-Sharing macht das Modell "strukturierter"
   - Filter sind interpretierbar (können visualisiert werden)

5. **Der entscheidende Lerneffekt:**
   - **8×8 Bilder:** MLP konkurrenzfähig
   - **28×28 MNIST:** CNN wäre deutlich besser
   - **224×224 ImageNet:** CNN unverzichtbar, MLP praktisch unmöglich

**Kernbotschaft:** 
- **Die Wahl der Architektur hängt von der Bildgröße ab**
- **Bei sehr kleinen Bildern können MLPs mithalten** - der CNN-Vorteil kommt erst bei "echten" Bildgrößen zum Tragen
- **Parameter-Effizienz allein macht noch keinen großen Unterschied** - die räumliche Komplexität muss groß genug sein
- **Dies erklärt, warum CNNs erst mit größeren Datensätzen wie ImageNet wirklich den Durchbruch geschafft haben**

**Zum Experimentieren:**
- Vergleiche mit MNIST (28×28) - dort wird der CNN-Vorteil deutlicher
- Teste mit verzerrten oder gedrehten 8×8 Bildern
- Probiere größere CNN-Filter (5×5) bei gleicher Parameter-Anzahl
- Implementiere Data Augmentation und schaue, welches Modell mehr profitiert